In [1]:
sc.install_pypi_package("numpy")
sc.install_pypi_package("matplotlib")
sc.install_pypi_package("seaborn")
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1749498710826_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1749498710826_0006/container_1749498710826_0006_01_000001/tmp/spark-a5e2dffe-2b79-46c0-be82-63d3140612e4
    Can't uninstall 'python-dateutil'. No files were found to uninstall.




ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.23.11 requires python-dateutil<=2.9.0,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import matplotlib.pyplot as plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName('FraudDetectionModeling') \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Step 2: Load processed parquet data from S3
data_path = 's3://projectamar/fraud-detection/output/processed_data/'
df = spark.read.parquet(data_path)
df.printSchema()
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- transaction_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- sender_account: string (nullable = true)
 |-- receiver_account: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- merchant_category: string (nullable = true)
 |-- location: string (nullable = true)
 |-- device_used: string (nullable = true)
 |-- is_fraud: integer (nullable = true)
 |-- fraud_type: string (nullable = true)
 |-- time_since_last_transaction: double (nullable = true)
 |-- spending_deviation_score: double (nullable = true)
 |-- velocity_score: integer (nullable = true)
 |-- geo_anomaly_score: double (nullable = true)
 |-- payment_channel: string (nullable = true)
 |-- ip_address: string (nullable = true)
 |-- device_hash: string (nullable = true)

+--------------+--------------------+--------------+----------------+-------+----------------+-----------------+--------+-----------+--------+----------------+-------

In [5]:
df = df.dropna().filter(col('amount') > 0).withColumn('is_fraud', col('is_fraud').cast('int'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
feature_cols = [
    'amount',
    'time_since_last_transaction',
    'spending_deviation_score',
    'velocity_score',
    'geo_anomaly_score'
]
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
df_features = assembler.transform(df).select('features', 'is_fraud')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
train_data, test_data = df_features.randomSplit([0.8, 0.2], seed=42)
print(f'Training data count: {train_data.count()}')
print(f'Testing data count: {test_data.count()}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Training data count: 143664
Testing data count: 35889

In [8]:
lr = LogisticRegression(labelCol='is_fraud', featuresCol='features')
lr_model = lr.fit(train_data)
print('Model trained successfully!')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model trained successfully!

In [9]:
predictions = lr_model.transform(test_data)
predictions.select('features', 'is_fraud', 'prediction', 'probability').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+----------+-----------+
|            features|is_fraud|prediction|probability|
+--------------------+--------+----------+-----------+
|[0.01,-8068.72086...|       1|       1.0|  [0.0,1.0]|
|[0.01,-7796.69908...|       1|       1.0|  [0.0,1.0]|
|[0.01,-7646.14419...|       1|       1.0|  [0.0,1.0]|
|[0.01,-7523.30761...|       1|       1.0|  [0.0,1.0]|
|[0.01,-7204.08469...|       1|       1.0|  [0.0,1.0]|
+--------------------+--------+----------+-----------+
only showing top 5 rows

In [10]:
binary_evaluator = BinaryClassificationEvaluator(labelCol='is_fraud')
auc = binary_evaluator.evaluate(predictions)
print(f'Area Under ROC: {auc}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Area Under ROC: 1.0

In [11]:
pred_pd = predictions.select('prediction', 'is_fraud').toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
import os
import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Step 9: Visualization (fraud vs non-fraud prediction count)
pred_pd = predictions.select('prediction', 'is_fraud').toPandas()
fraud_pred_counts = pred_pd.groupby(['is_fraud', 'prediction']).size().unstack().fillna(0)
fraud_pred_counts.plot(kind='bar', stacked=True, colormap='viridis')
plt.title('Fraud vs Non-Fraud Predictions')
plt.xlabel('Actual Label')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.legend(title='Predicted')
plt.tight_layout()
os.makedirs('/tmp', exist_ok=True)
plt.savefig('/tmp/fraud_prediction_plot.png')
plt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
import matplotlib.pyplot as plt
import seaborn as sns
import boto3

# Sample and convert to pandas
sample_df = df.select(
    "amount", "velocity_score", "geo_anomaly_score",
    "time_since_last_transaction", "spending_deviation_score"
).dropna().sample(False, 0.01).toPandas()

# Create and save pairplot
sns.pairplot(sample_df)
plt.suptitle("Fraud Detection Metrics Pairwise Plot", y=1.02)
plt.savefig("/tmp/pairplot.png")

# Upload to S3
s3 = boto3.client("s3")
s3.upload_file("/tmp/pairplot.png", "projectamar", "projectamar/output/pairplot.png")

print("Pairplot saved to s3://projectamar/output/pairplot.png")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pairplot saved to s3://projectamar/output/pairplot.png